In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 
import spacy

In [5]:
# stop_words_list = stopwords.words('english')
with open("stop_words") as file:
    stop_words_list = [line.strip() for line in file]


# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
spacy_lemmatizer = spacy.load('en', disable=['parser', 'ner'])

file_name_list = ["amazon.csv","apple.csv","google.csv","facebook.csv","uber.csv"]

## Process Common Crawl

In [7]:
for file_name in file_name_list:

    data = pd.read_csv('data_files/cc/'+file_name,names=['content'],header=None)
    print(data.shape)
    with open('processed_data/cc/' + file_name, 'w') as f:
        for index, row in data.iterrows():
            rowdata = row["content"]
            file_clear = re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", " ", rowdata.lower())
            file_lem = spacy_lemmatizer(file_clear)
            file_lem = " ".join([token.lemma_ for token in file_lem])
            tokens = nltk.word_tokenize(file_lem)
            filtered_words = [word for word in tokens if word not in stop_words_list]
            for item in filtered_words:
                f.write("%s " % item)
            f.write("\n ")
    f.close()

(120, 1)
(120, 1)
(120, 1)
(120, 1)
(120, 1)


## process Tweets

In [43]:
sample_size = 30000

for file_name in file_name_list:
    
    data = pd.read_csv('data_files/twitter/'+file_name)
    data['status_id'] = data['status_id'].str.strip()
    data.drop_duplicates(subset='status_id', keep = 'last', inplace = True)
    print(data.shape)
    dt = data.sample(n = sample_size)
    print(dt.shape)
    print('----------')
    
    with open('processed_data/twitter/' + file_name, 'w') as f:
        for index, row in dt.iterrows():
                rowdata = row["text"]
                file_clear = re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", " ", rowdata.lower())
                file_lem = spacy_lemmatizer(file_clear)
                file_lem = " ".join([token.lemma_ for token in file_lem])
                tokens = nltk.word_tokenize(file_lem)
                filtered_words = [word for word in tokens if word not in stop_words_list]
                for item in filtered_words:
                    f.write("%s " % item)
                f.write("\n ")
    f.close()
    



(35610, 88)
(30000, 88)
----------
(32804, 88)
(30000, 88)
----------
(35444, 88)
(30000, 88)
----------
(35997, 88)
(30000, 88)
----------
(32273, 88)
(30000, 88)
----------


## Process Nytimes data

In [42]:
sample_size = 300
for file_name in file_name_list:
    article_df = pd.read_csv('data_files/nyt/' + file_name)
    article_df['id'] = article_df['id'].str.strip()
    article_df.drop_duplicates(subset='id', keep = 'last', inplace = True)
    article_df = article_df.dropna()
    # article_df.sort_values("id", inplace = True) 
    print(article_df.shape)
    dt = article_df.sample(n = sample_size)
    print(dt.shape)
    print('----------')
    
    with open('processed_data/nyt/' + file_name, 'w') as f:
        for index, row in dt.iterrows():
                rowdata = row["content"]
                file_clear = re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)", " ", rowdata.lower())
                file_lem = spacy_lemmatizer(file_clear)
                file_lem = " ".join([token.lemma_ for token in file_lem])
                tokens = nltk.word_tokenize(file_lem)
                filtered_words = [word for word in tokens if word not in stop_words_list]
                for item in filtered_words:
                    f.write("%s " % item)
                f.write("\n ")
    f.close()
    

(950, 3)
(300, 3)
(457, 3)
(300, 3)
(1005, 3)
(300, 3)
(2104, 3)
(300, 3)
(307, 3)
(300, 3)


## Sort and get data

In [26]:
file_name = "uber"
# data = pd.read_csv('mr_output/nyt/'+file_name, sep='\t',header=None, names=['word','count'])
data = pd.read_csv('mr_output/cc/'+file_name, sep='\t',header=None, names=['word','count'])

In [27]:
data.sort_values(by=['count'], ascending=False).head(20)

,word,count
1216,company,630
4703,people,353
4074,million,343
6671,uber,269
667,billion,255
5728,service,254
7203,zuckerberg,253
3919,mark,250
3479,jonathan,234
7197,zittrain,233
